### Homework 2

Chris Dailey

christopher.dailey@gmail.com

W261-4

## Initial HDFS setup and testing

In [1]:
%%writefile wordcount.txt
hello hi hi hallo
bonjour hola hi ciao
nihao konnichiwa ola
hola nihao hello

Writing wordcount.txt


### Mapper

In [2]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print(word+", 1")

Writing mapper.py


In [34]:
!chmod +x mapper.py

In [4]:
!cat wordcount.txt | python mapper.py

hello, 1
hi, 1
hi, 1
hallo, 1
bonjour, 1
hola, 1
hi, 1
ciao, 1
nihao, 1
konnichiwa, 1
ola, 1
hola, 1
nihao, 1
hello, 1


### Reducer

In [15]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split(",")

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print(current_word + "\t" + str(current_count))
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print(current_word + "\t" + str(current_count))

Overwriting reducer.py


In [35]:
!chmod +x reducer.py

In [17]:
!echo "foo foo quux labs foo bar quux" | python mapper.py | sort -k1,1 | python reducer.py

bar	1
foo	3
labs	1
quux	2


In [32]:
!hadoop/bin/hdfs dfs -rm /user/ubuntu/wordcountOutput/*
!hadoop/bin/hdfs dfs -rmdir /user/ubuntu/wordcountOutput

rm: `/user/ubuntu/wordcountOutput/*': No such file or directory


In [19]:
hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar -file ~/repos/261/HW2/mapper.py -mapper ~/repos/261/HW2/mapper.py -file ~/repos/261/HW2/reducer.py -reducer ~/repos/261/HW2/reducer.py -input /user/ubuntu/wordcount.txt -output /user/ubuntu/wordcountOutput

/bin/sh: 1: hdfs: not found


In [27]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/wordcountOutput/part-00000

bonjour	1
ciao	1
hallo	1
hello	2
hi	3
hola	2
konnichiwa	1
nihao	2
ola	1


In [36]:
!hadoop/bin/hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar -mapper ~/repos/261/HW2/mapper.py -reducer ~/repos/261/HW2/reducer.py -input /user/ubuntu/wordcount.txt -output /user/ubuntu/wordcountOutput

packageJobJar: [/tmp/hadoop-unjar3869171142529360995/] [] /tmp/streamjob4695714299996201914.jar tmpDir=null
16/01/24 18:10:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 18:10:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 18:10:19 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/24 18:10:19 INFO mapreduce.JobSubmitter: number of splits:2
16/01/24 18:10:19 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453569821305_0011
16/01/24 18:10:19 INFO impl.YarnClientImpl: Submitted application application_1453569821305_0011
16/01/24 18:10:19 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453569821305_0011/
16/01/24 18:10:19 INFO mapreduce.Job: Running job: job_1453569821305_0011
16/01/24 18:10:24 INFO mapreduce.Job: Job job_1453569821305_0011 running in uber mode : false
16/01/24 18:10:24 INFO mapreduce.Job:  map 0% reduce 0%
16/01/24 18:10:31 INFO ma

In [37]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/wordcountOutput/part-00000

bonjour	1
ciao	1
hallo	1
hello	2
hi	3
hola	2
konnichiwa	1
nihao	2
ola	1


## HW 2.1
Sort in Hadoop MapReduce

In [38]:
!hadoop/bin/hdfs dfs -mkdir /user/ubuntu/sort

In [54]:
import random as rnd

integers = open("integers.txt", 'w')

for i in range(10000):
    integers.write(str(int(rnd.random()*5000))+", \n")
integers.close()
print("done")

done


In [55]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    integer, blank = line.split(",")
    print(str(integer))

Overwriting mapper.py


In [56]:
!chmod +x mapper.py

In [84]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
integers = []
for line in sys.stdin:
    # remove leading and trailing whitespace
    integers.append(int(line))

print("Ten biggest:")
for i in range(10):
    print(integers[i])
print("\n\n")
print("Ten smallest:")
for i in range(len(integers)-10, len(integers)):
    print(integers[i])

Overwriting reducer.py


In [85]:
!chmod +x reducer.py

In [87]:
!hadoop/bin/hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options=-k1nr \
-mapper /home/ubuntu/repos/261/HW2/mapper.py \
-reducer /home/ubuntu/repos/261/HW2/reducer.py \
-input /user/ubuntu/sort/integers.txt \
-output /user/ubuntu/sort/output

packageJobJar: [/tmp/hadoop-unjar7102658785291136101/] [] /tmp/streamjob5701401045672634559.jar tmpDir=null
16/01/24 21:11:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 21:11:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 21:11:33 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/24 21:11:33 INFO mapreduce.JobSubmitter: number of splits:2
16/01/24 21:11:33 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453569821305_0018
16/01/24 21:11:34 INFO impl.YarnClientImpl: Submitted application application_1453569821305_0018
16/01/24 21:11:34 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453569821305_0018/
16/01/24 21:11:34 INFO mapreduce.Job: Running job: job_1453569821305_0018
16/01/24 21:11:39 INFO mapreduce.Job: Job job_1453569821305_0018 running in uber mode : false
16/01/24 21:11:39 INFO mapreduce.Job:  map 0% reduce 0%
16/01/24 21:11:46 INFO ma

In [88]:
!hadoop/bin/hdfs dfs -ls /user/ubuntu/sort/output

Found 2 items
-rw-r--r--   1 ubuntu supergroup          0 2016-01-24 21:11 /user/ubuntu/sort/output/_SUCCESS
-rw-r--r--   1 ubuntu supergroup        125 2016-01-24 21:11 /user/ubuntu/sort/output/part-00000


In [89]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/sort/output/part-00000

Ten biggest:	
4998	
4998	
4998	
4997	
4996	
4996	
4996	
4995	
4995	
4995	
	
	
	
Ten smallest:	
4	
4	
4	
3	
3	
2	
2	
1	
1	
0	


In [86]:
!hadoop/bin/hdfs dfs -rm /user/ubuntu/sort/output/*
!hadoop/bin/hdfs dfs -rmdir /user/ubuntu/sort/output

16/01/24 21:11:26 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/sort/output/_SUCCESS
16/01/24 21:11:26 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/sort/output/part-00000


## HW 2.2

Word count

In [90]:
%%writefile mapper.py
#!/usr/bin/python
import sys, re
# input comes from STDIN (standard input)
findword = sys.argv[1]
WORD_RE = re.compile(r"[\w']+")
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    for word in [x.lower() for x in WORD_RE.findall(line)]:
        if word == findword:
            print(word+", 1")

Overwriting mapper.py


In [91]:
!chmod +x mapper.py

In [106]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)

currentWord= None
currentCount = 0
for line in sys.stdin:
    word, count = line.split(",")
    if currentWord is None:
        currentWord = word.lower()
    if word.lower() == currentWord:
        currentCount += int(count.rstrip())
    else:
        print(currentWord + " " + str(currentCount))
        currentWord = word
        currentCount = 0
print(currentWord + " " + str(currentCount))
    

Overwriting reducer.py


In [107]:
!chmod +x reducer.py

In [108]:
!hadoop/bin/hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper "/home/ubuntu/repos/261/HW2/mapper.py assistance" \
-reducer /home/ubuntu/repos/261/HW2/reducer.py \
-input /user/ubuntu/wordcount/enronemail_1h.txt \
-output /user/ubuntu/wordcountOutput

packageJobJar: [/tmp/hadoop-unjar4096940200557490376/] [] /tmp/streamjob6170588845171672975.jar tmpDir=null
16/01/24 22:33:49 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 22:33:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 22:33:50 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/24 22:33:50 INFO mapreduce.JobSubmitter: number of splits:2
16/01/24 22:33:50 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453569821305_0022
16/01/24 22:33:50 INFO impl.YarnClientImpl: Submitted application application_1453569821305_0022
16/01/24 22:33:50 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453569821305_0022/
16/01/24 22:33:50 INFO mapreduce.Job: Running job: job_1453569821305_0022
16/01/24 22:33:55 INFO mapreduce.Job: Job job_1453569821305_0022 running in uber mode : false
16/01/24 22:33:55 INFO mapreduce.Job:  map 0% reduce 0%
16/01/24 22:34:01 INFO ma

In [109]:
!hadoop/bin/hdfs dfs -ls /user/ubuntu/wordcountOutput

Found 2 items
-rw-r--r--   1 ubuntu supergroup          0 2016-01-24 22:34 /user/ubuntu/wordcountOutput/_SUCCESS
-rw-r--r--   1 ubuntu supergroup         15 2016-01-24 22:34 /user/ubuntu/wordcountOutput/part-00000


In [110]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/wordcountOutput/part-00000

assistance 10	


In [105]:
!hadoop/bin/hdfs dfs -rm /user/ubuntu/wordcountOutput/*
!hadoop/bin/hdfs dfs -rmdir /user/ubuntu/wordcountOutput

rm: `/user/ubuntu/wordcountOutput/*': No such file or directory


## HW 2.2.1

Finding the top ten most common tokens

In [140]:
%%writefile mapper.py
#!/usr/bin/python
import sys, re
# input comes from STDIN (standard input)
WORD_RE = re.compile(r"[\w']+")
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    for word in [x.lower() for x in WORD_RE.findall(line)]:
        print(word+", 1")

Overwriting mapper.py


In [169]:
%%writefile reducer.py
#!/usr/bin/python
import sys, operator
# input comes from STDIN (standard input)

counts = {}

currentWord= None
currentCount = 0
for line in sys.stdin:
    word, count = line.split(",")
    if currentWord is None:
        currentWord = word.lower()
    if word.lower() == currentWord:
        currentCount += int(count.rstrip())
    else:
        counts[currentWord] = currentCount
        currentWord = word
        currentCount = int(count.rstrip())
counts[currentWord] = currentCount


print("Top ten most common tokens:")
for key, value in sorted(counts.items(), key=operator.itemgetter(1), reverse=True)[:10]:
    print(key + " " + str(value))

Overwriting reducer.py


In [170]:
%%writefile test.txt
hat, 10
hat, 5
cat, 6

Overwriting test.txt


In [171]:
!cat test.txt | python reducer.py

Top ten most common tokens:
hat 15
cat 6


In [172]:
!hadoop/bin/hdfs dfs -rm /user/ubuntu/wordcountOutput/*
!hadoop/bin/hdfs dfs -rmdir /user/ubuntu/wordcountOutput

16/01/24 23:23:35 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/wordcountOutput/_SUCCESS
16/01/24 23:23:35 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/wordcountOutput/part-00000


In [173]:
!hadoop/bin/hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW2/mapper.py \
-reducer /home/ubuntu/repos/261/HW2/reducer.py \
-input /user/ubuntu/wordcount/enronemail_1h.txt \
-output /user/ubuntu/wordcountOutput

packageJobJar: [/tmp/hadoop-unjar1833401051588499007/] [] /tmp/streamjob7021344597045634453.jar tmpDir=null
16/01/24 23:23:42 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 23:23:42 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 23:23:42 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/24 23:23:42 INFO mapreduce.JobSubmitter: number of splits:2
16/01/24 23:23:43 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453569821305_0025
16/01/24 23:23:43 INFO impl.YarnClientImpl: Submitted application application_1453569821305_0025
16/01/24 23:23:43 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453569821305_0025/
16/01/24 23:23:43 INFO mapreduce.Job: Running job: job_1453569821305_0025
16/01/24 23:23:48 INFO mapreduce.Job: Job job_1453569821305_0025 running in uber mode : false
16/01/24 23:23:48 INFO mapreduce.Job:  map 0% reduce 0%
16/01/24 23:23:54 INFO ma

In [174]:
!hadoop/bin/hdfs dfs -ls /user/ubuntu/wordcountOutput

Found 2 items
-rw-r--r--   1 ubuntu supergroup          0 2016-01-24 23:23 /user/ubuntu/wordcountOutput/_SUCCESS
-rw-r--r--   1 ubuntu supergroup        116 2016-01-24 23:23 /user/ubuntu/wordcountOutput/part-00000


In [175]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/wordcountOutput/part-00000

Top ten most common tokens:	
the 1247	
to 963	
and 668	
of 566	
a 542	
you 432	
in 417	
your 394	
ect 382	
for 373	


## HW 2.3
Multinomial Naive Bayes without smoothing

In [293]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

WORD_RE = re.compile(r"[\w']+")
for line in sys.stdin:
    components = line.split('\t')
    if len(components) < 4:
        continue
    ID = components[0]
    flag = components[1]
    text = components[2] + " " + components[3]
    word_counts = {}
    #do a little in-mapper combining
    for word in WORD_RE.findall(text):
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1
    for word, count in word_counts.items():
        print(ID + "\t" + str(flag) + "\t" + word + "\t" + str(count))

Overwriting mapper.py


In [234]:
!cat enronemail_1h.txt | python mapper.py

0001.1999-12-10.farmer	0	pictures	1
0001.1999-12-10.farmer	0	christmas	1
0001.1999-12-10.farmer	0	farm	1
0001.1999-12-10.farmer	0	NA	1
0001.1999-12-10.farmer	0	tree	1
0001.1999-12-10.kaminski	0	thank	1
0001.1999-12-10.kaminski	0	you	1
0001.1999-12-10.kaminski	0	rankings	1
0001.1999-12-10.kaminski	0	re	1
0001.2000-01-17.beck	0	further	1
0001.2000-01-17.beck	0	walton	1
0001.2000-01-17.beck	0	styles	1
0001.2000-01-17.beck	0	ready	2
0001.2000-01-17.beck	0	need	1
0001.2000-01-17.beck	0	by	2
0001.2000-01-17.beck	0	christine	1
0001.2000-01-17.beck	0	up	3
0001.2000-01-17.beck	0	have	4
0001.2000-01-17.beck	0	norma	1
0001.2000-01-17.beck	0	we've	1
0001.2000-01-17.beck	0	only	1
0001.2000-01-17.beck	0	there	1
0001.2000-01-17.beck	0	on	6
0001.2000-01-17.beck	0	benefit	1
0001.2000-01-17.beck	0	jones	1
0001.2000-01-17.beck	0	available	2
0001.2000-01-17.beck	0	later	1
0001.2000-01-17.beck	0	gracie	2
0001.2000-01-17.beck	0	even	1
0001.2000-01-17.beck	0	results	1
0001.2000-01-17.beck	0	shall	1
0001.2000

In [294]:
%%writefile reducer.py
#!/usr/bin/python
import sys, re
from math import log

#for calculating priors
emails = set()
spams = set()
#for calculating cond probs
words_in_corpus = set()
spam_counts = {}
ham_counts = {}

WORD_RE = re.compile(r"[\w']+")
for line in sys.stdin:
    splits = re.split('\t', line)
    if len(splits) < 4:
        continue
    emailID = splits[0]
    label = splits[1]
    word = splits[2]
    count = int(splits[3].rstrip())

    
    emails.add(emailID)
    words_in_corpus.add(word)
    
    #if the email is spam, add it to the list of spams
    #and add all the words in it to the spam counts
    if (label == '1'):
        spams.add(emailID)
        if (word not in spam_counts):
            spam_counts[word] = count
        else:
            spam_counts[word] += count
    #if it's not spam, add the words to the ham counts
    else:
        if (word not in ham_counts):
            ham_counts[word] = count
        else:
            ham_counts[word] += count

            
prior_spam = 1.*len(spams)/len(emails)
prior_ham = 1.*(len(emails) - len(spams))/len(emails)
condprob_ham = {}
condprob_spam = {}

#with no smoothing, words that do not occur in class c do not contribute to the denominator 
#of the conditional probability.  Words that do not occur in class c have a probability of 0
#which will have to be accounted for in the classification stage.
for word in words_in_corpus:
    if word in spam_counts:
        condprob_spam[word] = 1.*(spam_counts[word])/sum(spam_counts.values())
    else:
        condprob_spam[word] = 0
    if word in ham_counts:
        condprob_ham[word] = 1.*(ham_counts[word])/(sum(ham_counts.values()))
        
    else:
        condprob_ham[word] = 0
        
print(prior_spam)
print(prior_ham)
print(str(len(condprob_spam)))
for word, condprob in condprob_spam.items():
    print(word + ":" + str(condprob))
print(str(len(condprob_ham)))
for word, condprob in condprob_ham.items():
    print(word + ":" + str(condprob))

Overwriting reducer.py


In [251]:
!cat enronemail_1h.txt | python mapper.py | sort | python reducer.py

Found a hamword: christmas
Found a hamword: farm
Found a hamword: NA
Found a hamword: pictures
Found a hamword: tree
Found a hamword: rankings
Found a hamword: re
Found a hamword: thank
Found a hamword: you
Found a hamword: 01
Found a hamword: 03
Found a hamword: 06
Found a hamword: 10
Found a hamword: 12
Found a hamword: 1
Found a hamword: 17
Found a hamword: 2000
Found a hamword: 22
Found a hamword: 3
Found a hamword: 33597
Found a hamword: 44
Found a hamword: 5
Found a hamword: 560
Found a hamword: 6
Found a hamword: 7394
Found a hamword: 8
Found a hamword: 9
Found a hamword: a
Found a hamword: all
Found a hamword: allen
Found a hamword: also
Found a hamword: am
Found a hamword: an
Found a hamword: and
Found a hamword: any
Found a hamword: appropriate
Found a hamword: are
Found a hamword: armstrong
Found a hamword: as
Found a hamword: ask
Found a hamword: asking
Found a hamword: at
Found a hamword: attached
Found a hamword: attend
Found a hamword: attendance
Found a hamword: attendi

In [296]:
!hadoop/bin/hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW2/mapper.py \
-reducer /home/ubuntu/repos/261/HW2/reducer.py \
-input /user/ubuntu/wordcount/enronemail_1h.txt \
-output /user/ubuntu/NBOutput

packageJobJar: [/tmp/hadoop-unjar4004979936813764798/] [] /tmp/streamjob5608469063708143321.jar tmpDir=null
16/01/26 03:42:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/26 03:42:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/26 03:42:07 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/26 03:42:07 INFO mapreduce.JobSubmitter: number of splits:2
16/01/26 03:42:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453569821305_0040
16/01/26 03:42:07 INFO impl.YarnClientImpl: Submitted application application_1453569821305_0040
16/01/26 03:42:07 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453569821305_0040/
16/01/26 03:42:07 INFO mapreduce.Job: Running job: job_1453569821305_0040
16/01/26 03:42:12 INFO mapreduce.Job: Job job_1453569821305_0040 running in uber mode : false
16/01/26 03:42:12 INFO mapreduce.Job:  map 0% reduce 0%
16/01/26 03:42:20 INFO ma

In [262]:
!hadoop/bin/hdfs dfs -ls /user/ubuntu/NBOutput

Found 2 items
-rw-r--r--   1 ubuntu supergroup          0 2016-01-26 00:26 /user/ubuntu/NBOutput/_SUCCESS
-rw-r--r--   1 ubuntu supergroup     218351 2016-01-26 00:26 /user/ubuntu/NBOutput/part-00000


In [263]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/NBOutput/part-00000

0.438775510204	
0.561224489796	
5433	
stock:0.000109781534746	
limited:0.000494016906356	
entergyr:0	
paul:0.000109781534746	
believed:0	
child:0.000109781534746	
dynamic:0	
knelt:5.48907673729e-05	
yellow:5.48907673729e-05	
four:0.000274453836865	
protest:0	
sleep:0.000109781534746	
invovled:0	
'mperkins:0	
railing:5.48907673729e-05	
appetite:0.000109781534746	
hate:0.000219563069492	
forget:5.48907673729e-05	
whose:5.48907673729e-05	
fronts:0	
chinese:5.48907673729e-05	
granting:5.48907673729e-05	
electricity:0	
'chuck:0	
originality:5.48907673729e-05	
bailout:0	
koromah:0.000109781534746	
swap:0.000109781534746	
whatsoever:0.000164672302119	
under:0.000494016906356	
pamela:0	
sorry:0	
pride:0	
homemakers:5.48907673729e-05	
349:0.000109781534746	
overpaying:5.48907673729e-05	
merchant:0.000164672302119	
plants:0	
updated:5.48907673729e-05	
collaborate:0	
legislators:0	
risk:0.000164672302119	
regional:0	
distort:5.48907673729e-05	
systemslogical:5.48907673729e-05	
crenshaw:0	
every:0

In [295]:
!hadoop/bin/hdfs dfs -rm /user/ubuntu/NBOutput/*
!hadoop/bin/hdfs dfs -rmdir /user/ubuntu/NBOutput

16/01/26 03:42:01 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/NBOutput/_SUCCESS
16/01/26 03:42:01 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/NBOutput/part-00000


In [265]:
!hadoop/bin/hdfs dfs -get /user/ubuntu/NBOutput/part-00000 ./probs.txt

16/01/26 00:33:06 WARN hdfs.DFSClient: DFSInputStream has been closed already


In [324]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re
from math import log

WORD_RE = re.compile(r"[\w']+")

probs = open(sys.argv[1], 'r')
#probs = open("probs.txt", 'r')
prior_spam = float(probs.readline().rstrip())
prior_ham = float(probs.readline().rstrip())

condprobs_spam = {}
condprobs_ham = {}

ham_ignores = 0
spam_ignores = 0

num_condprobs_spam = int(probs.readline().rstrip())
for i in range(num_condprobs_spam):
    splits = re.split(':', probs.readline().rstrip())
    if len(splits) != 2:
        continue
    condprobs_spam[splits[0]] = float(splits[1])
    
num_condprobs_ham = int(probs.readline().rstrip())
for i in range(num_condprobs_ham):
    splits = re.split(':', probs.readline().rstrip())
    if len(splits) != 2:
        continue
    condprobs_ham[splits[0]] = float(splits[1])
    
    
email_count = 0
correct_guesses = 0
for line in sys.stdin:
    email_count += 1
    splits = re.split("\t", line)
    if len(splits) != 4:
        continue
    emailID = splits[0]
    label = splits[1]
    text = splits[2] + " " + splits[3]
    local_prob_spam = log(prior_spam)
    local_prob_ham = log(prior_ham)
    classification = 0
    printed = False
    for word in WORD_RE.findall(text):
        if condprobs_ham[word] > 0:
            local_prob_ham += log(condprobs_ham[word])
        else:
            print(emailID + "\t" + str(label) + "\t" + str(1))
            if label == '1':
                correct_guesses += 1
            ham_ignores += 1
            printed = True
            break
        if condprobs_spam[word] > 0:
            local_prob_spam += log(condprobs_spam[word])
        else:
            print(emailID + "\t" + str(label) + "\t" + str(0))
            if label == '0':
                correct_guesses += 1
            spam_ignores += 1
            printed = True
            break
    if printed:
        continue
    if local_prob_spam < local_prob_ham:
        classification = 1
    print(emailID + "\t" + str(label) + "\t" + str(classification))
print("")   
print("Accuracy: " + str(1.*correct_guesses/email_count))
print("Ham zeros: " + str(ham_ignores))
print("Spam zeros: " + str(spam_ignores))

Overwriting mapper.py


In [325]:
!cat enronemail_1h.txt | python mapper.py probs.txt

0001.1999-12-10.farmer	0	0
0001.1999-12-10.kaminski	0	0
0001.2000-01-17.beck	0	0
0001.2001-02-07.kitchen	0	0
0001.2001-04-02.williams	0	0
0002.1999-12-13.farmer	0	0
0002.2001-02-07.kitchen	0	0
0002.2001-05-25.SA_and_HP	1	1
0002.2003-12-18.GP	1	1
0002.2004-08-01.BG	1	1
0003.1999-12-10.kaminski	0	0
0003.1999-12-14.farmer	0	0
0003.2000-01-17.beck	0	0
0003.2001-02-08.kitchen	0	0
0003.2003-12-18.GP	1	1
0003.2004-08-01.BG	1	1
0004.1999-12-10.kaminski	0	0
0004.1999-12-14.farmer	0	0
0004.2001-04-02.williams	0	0
0004.2001-06-12.SA_and_HP	1	1
0004.2004-08-01.BG	1	1
0005.1999-12-12.kaminski	0	0
0005.1999-12-14.farmer	0	0
0005.2000-06-06.lokay	0	0
0005.2001-02-08.kitchen	0	0
0005.2001-06-23.SA_and_HP	1	1
0005.2003-12-18.GP	1	1
0006.1999-12-13.kaminski	0	0
0006.2001-02-08.kitchen	0	0
0006.2001-04-03.williams	0	0
0006.2001-06-25.SA_and_HP	1	1
0006.2003-12-18.GP	1	1
0006.2004-08-01.BG	1	1
0007.1999-12-13.kaminski	0	0
0007.1999-12-14.farmer	0	0
0007.2000-01-17.beck	0	0
0007.2001-02-09.kitchen	0	0
0007

In [327]:
!hadoop/bin/hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper "/home/ubuntu/repos/261/HW2/mapper.py /home/ubuntu/repos/261/HW2/probs.txt" \
-file /home/ubuntu/repos/261/HW2/probs.txt \
-input /user/ubuntu/wordcount/enronemail_1h.txt \
-output /user/ubuntu/NBOutputStage2

16/01/26 04:17:06 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/home/ubuntu/repos/261/HW2/probs.txt, /tmp/hadoop-unjar4490023069186457642/] [] /tmp/streamjob6240921243096823619.jar tmpDir=null
16/01/26 04:17:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/26 04:17:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/26 04:17:07 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/26 04:17:07 INFO mapreduce.JobSubmitter: number of splits:2
16/01/26 04:17:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453569821305_0046
16/01/26 04:17:08 INFO impl.YarnClientImpl: Submitted application application_1453569821305_0046
16/01/26 04:17:08 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453569821305_0046/
16/01/26 04:17:08 INFO mapreduce.Job: Running job: job_1453569821305_0046
16/01/26 04:17:13 INFO 

In [326]:
!hadoop/bin/hdfs dfs -rm /user/ubuntu/NBOutputStage2/*
!hadoop/bin/hdfs dfs -rmdir /user/ubuntu/NBOutputStage2

rm: `/user/ubuntu/NBOutputStage2/*': No such file or directory


In [328]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/NBOutputStage2/part-00000

	
	
0001.1999-12-10.farmer	0	0
0001.1999-12-10.kaminski	0	0
0001.2000-01-17.beck	0	0
0001.2001-02-07.kitchen	0	0
0001.2001-04-02.williams	0	0
0002.1999-12-13.farmer	0	0
0002.2001-02-07.kitchen	0	0
0002.2001-05-25.SA_and_HP	1	1
0002.2003-12-18.GP	1	1
0002.2004-08-01.BG	1	1
0003.1999-12-10.kaminski	0	0
0003.1999-12-14.farmer	0	0
0003.2000-01-17.beck	0	0
0003.2001-02-08.kitchen	0	0
0003.2003-12-18.GP	1	1
0003.2004-08-01.BG	1	1
0004.1999-12-10.kaminski	0	0
0004.1999-12-14.farmer	0	0
0004.2001-04-02.williams	0	0
0004.2001-06-12.SA_and_HP	1	1
0004.2004-08-01.BG	1	1
0005.1999-12-12.kaminski	0	0
0005.1999-12-14.farmer	0	0
0005.2000-06-06.lokay	0	0
0005.2001-02-08.kitchen	0	0
0005.2001-06-23.SA_and_HP	1	1
0005.2003-12-18.GP	1	1
0006.1999-12-13.kaminski	0	0
0006.2001-02-08.kitchen	0	0
0006.2001-04-03.williams	0	0
0006.2001-06-25.SA_and_HP	1	1
0006.2003-12-18.GP	1	1
0006.2004-08-01.BG	1	1
0007.1999-12-13.kaminski	0	0
0007.1999-12-14.farmer	0	0
0007.2000-01-17.beck	0	0
0007.2001-02-09.kitchen	0	0


## HW 2.4

Multinomial Naive Bayes with Laplace smoothing

In [329]:
%%writefile mapper.py
#!/usr/bin/python

import sys, re

WORD_RE = re.compile(r"[\w']+")
for line in sys.stdin:
    components = line.split('\t')
    if len(components) < 4:
        continue
    ID = components[0]
    flag = components[1]
    text = components[2] + " " + components[3]
    word_counts = {}
    #do a little in-mapper combining
    for word in WORD_RE.findall(text):
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1
    for word, count in word_counts.items():
        print(ID + "\t" + str(flag) + "\t" + word + "\t" + str(count))

Overwriting mapper.py


In [330]:
%%writefile reducer.py
#!/usr/bin/python
import sys, re
from math import log

#for calculating priors
emails = set()
spams = set()
#for calculating cond probs
words_in_corpus = set()
spam_counts = {}
ham_counts = {}

WORD_RE = re.compile(r"[\w']+")
for line in sys.stdin:
    splits = re.split('\t', line)
    if len(splits) < 4:
        continue
    emailID = splits[0]
    label = splits[1]
    word = splits[2]
    count = int(splits[3].rstrip())

    
    emails.add(emailID)
    words_in_corpus.add(word)
    
    #if the email is spam, add it to the list of spams
    #and add all the words in it to the spam counts
    if (label == '1'):
        spams.add(emailID)
        if (word not in spam_counts):
            spam_counts[word] = count
        else:
            spam_counts[word] += count
    #if it's not spam, add the words to the ham counts
    else:
        if (word not in ham_counts):
            ham_counts[word] = count
        else:
            ham_counts[word] += count

            
prior_spam = 1.*len(spams)/len(emails)
prior_ham = 1.*(len(emails) - len(spams))/len(emails)
condprob_ham = {}
condprob_spam = {}

#laplace smoothing
for word in words_in_corpus:
    if word in spam_counts:
        condprob_spam[word] = 1.*(spam_counts[word] + 1)/(sum(spam_counts.values()) + len(words_in_corpus))
    else:
        condprob_spam[word] = 1.*(1)/(sum(spam_counts.values()) + len(words_in_corpus))
    if word in ham_counts:
        condprob_ham[word] = 1.*(ham_counts[word] + 1)/((sum(ham_counts.values())) + len(words_in_corpus))
    else:
        condprob_ham[word] = 1.*(1)/((sum(ham_counts.values())) + len(words_in_corpus))
        
print(prior_spam)
print(prior_ham)
print(str(len(condprob_spam)))
for word, condprob in condprob_spam.items():
    print(word + ":" + str(condprob))
print(str(len(condprob_ham)))
for word, condprob in condprob_ham.items():
    print(word + ":" + str(condprob))

Overwriting reducer.py


In [ ]:
!hadoop/bin/hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper /home/ubuntu/repos/261/HW2/mapper.py \
-reducer /home/ubuntu/repos/261/HW2/reducer.py \
-input /user/ubuntu/wordcount/enronemail_1h.txt \
-output /user/ubuntu/NBOutput